<a href="https://colab.research.google.com/github/vaishnavi-konda/youtube-transcript-summarizer/blob/main/Copy_of_Harsha_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture 
!pip install --upgrade yt_dlp
!pip install pydub SpeechRecognition ffmpeg ffprobe googletrans==3.1.0a0 transformers
!pip install rouge
import yt_dlp
import time
import re
import os
from pydub import AudioSegment
import speech_recognition as sr
import math
from tqdm import tqdm
from googletrans import Translator
from threading import Thread
!pip install transformers[torch]==4.3
!pip install packaging==21.3
!pip install torch
!pip install sentencepiece
import torch
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import T5Config
!pip install gtts
from gtts import gTTS
from IPython.display import Audio, display
from rouge import Rouge
import json

In [ ]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=5ac7bce53fcbe906a291a5862be2a52d68cf1b265a82c11c12e558fccb417bc0
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import anvil.server

anvil.server.connect("server_OGM6JCVQ6AH4Q4LREDA2GRAC-AQVTOKX2GAGASCF3")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
url=''
@anvil.server.callable
def input_text(yurl): 
  url = yurl
  ydl_opts={}

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
          info_dict = ydl.extract_info(url, download=False)
  video_title = info_dict['title']
  video_name = re.sub('[\\\\/*?:"<>|]', '', video_title)
  name = video_name
  ydl_opts = {
     'format': 'm4a/bestaudio/best',
         'noplaylist': True,
         'continue_dl': True,
         'outtmpl': f'./{name}.wav',
         'postprocessors': [{
             'key': 'FFmpegExtractAudio',
             'preferredcodec': 'wav',
             'preferredquality': '192',
         }],
         'geobypass':True,
         'ffmpeg_location':'/usr/bin/ffmpeg'
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      error_code = ydl.download(url)
  #Splitting audio
  class SplitWavAudioMubin():
      def __init__(self, folder, filename):
          self.folder = folder
          self.filename = filename
          self.filepath = folder + filename
        # print(self.filepath)
          self.audio = AudioSegment.from_wav(self.filepath)
    
      def get_duration(self):
          return self.audio.duration_seconds
    
      def single_split(self, from_min, to_min, split_filename):
          t1 = from_min * 60 * 1000
          t2 = to_min * 60 * 1000
          split_audio = self.audio[t1:t2]
          split_audio.export(split_filename, format="wav")
        
      def multiple_split(self, min_per_split):
          total_mins = math.ceil(self.get_duration() / 60)
          for i in range(0, total_mins, min_per_split):
              split_fn = str(i) + '_' + self.filename
              self.single_split(i, i+min_per_split, split_fn)
              if i == total_mins - min_per_split:
                  print('All splited successfully')
          print('>>> Video duration: ' + str(self.get_duration()))
        
  def split_audio(file_name):
        directory = "splitted files for: " + file_name
        os.mkdir(directory)
        split_wav = SplitWavAudioMubin("/content/", file_name)
        split_wav.multiple_split(min_per_split=1)

  file_name = "{}.wav.wav".format(video_name)
  split_audio(file_name)
  search_dir = "./"
  files = filter(os.path.isfile, os.listdir(search_dir))
  files = [os.path.join(search_dir, f) for f in files] # add path to each file
  files.sort(key=lambda x: os.path.getmtime(x))

  def speech_recognizer(files, frames, i):  ## This function recognizes speech in our WAV files
    texts = []
    recognizer = sr.Recognizer()

    for file in tqdm(files):
        with sr.AudioFile(file) as source:
            recorded_audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(
                recorded_audio,
                language="en-US"  ## Replace with language keyword
            )
            texts.append(text)
        except Exception as ex:
            print(ex)
    result = ""
    for text in texts:
        result += " " + text
    frames[i] = result
    return result

  def split_files(files, n_batches): ## This function split the files evenly between the threads we have.
    k, m = divmod(len(files), n_batches)
    return list(files[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n_batches))

  def main(n_batches=8, verbose=True): ## By default, the maximum capacity of threads supported in collab is 8
    threads = [None]*n_batches
    frames = [None]*n_batches
    batches = split_files(files, n_batches)
    start = 0
    for i in range(len(batches)):
        if i>0:
            start_index=len(batches[i-1])
        else:
            start_index = 0
        t = Thread(target=speech_recognizer, args=(batches[i], frames, i))
        threads[i] = t
        t.start()
    for t in threads:
        t.join()
    return frames
  
  frames1 = main()
  result1 = ""
  for text in frames1:
      result1 += " " + text
  print(result1)
  os.chdir("../")
  text_file = open("Transcription_"+ file_name[:-4] +".txt", "w")
  text_file.write(result1)
  text_file.close()
  model = T5ForConditionalGeneration.from_pretrained('t5-large')
  tokenizer = T5Tokenizer.from_pretrained('t5-large')
  device = torch.device('cpu')
  text=result1

  preprocessed_text =text.strip().replace('\n','')
  t5_input_text = 'summarize: ' + preprocessed_text
  ml=len(t5_input_text.split())
  tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=ml).to(device)
  summary_ids = model.generate(tokenized_text, min_length=30, max_length=200)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  
  #return summary      
  #This module is imported so that we can
  #play the converted audio
  #Provide the string to convert to speech
  tts = gTTS(summary)                     
  #save the string converted to speech as a .wav file    
  tts.save('final.wav') 
  sound_file = 'final.wav'
  display(Audio(sound_file, autoplay=True))
  return summary


In [ ]:
@anvil.server.callable
def eval(fref,summary):
  reference=fref
  rouge = Rouge()
  r_score=rouge.get_scores(summary, reference)
  print(json.dumps(r_score))
  return json.dumps(r_score)

In [ ]:
anvil.server.wait_forever()

[youtube] Extracting URL: https://youtu.be/QUTYxwTsbiM
[youtube] QUTYxwTsbiM: Downloading webpage
[youtube] QUTYxwTsbiM: Downloading android player API JSON
[youtube] Extracting URL: https://youtu.be/QUTYxwTsbiM
[youtube] QUTYxwTsbiM: Downloading webpage
[youtube] QUTYxwTsbiM: Downloading android player API JSON
[info] QUTYxwTsbiM: Downloading 1 format(s): 140
[dashsegments] Total fragments: 1
[download] Destination: ./Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav
[download] 100% of    3.27MiB in 00:00:00 at 5.91MiB/s               
[FixupM4a] Correcting container of "./Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav"
[ExtractAudio] Destination: ./Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav.wav
Deleting original file ./Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav (pass -k to keep)
All splited success



  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/1 [00:00<?, ?it/s]




0it [00:00, ?it/s]







0it [00:00, ?it/s]




0it [00:00, ?it/s]



100%|██████████| 1/1 [00:01<00:00,  1.43s/it]






100%|██████████| 1/1 [00:01<00:00,  1.84s/it]



100%|██████████| 1/1 [00:03<00:00,  3.47s/it]

100%|██████████| 1/1 [00:06<00:00,  6.13s/it]


100%|██████████| 1/1 [00:06<00:00,  6.28s/it]


  subscribe to Sonic octaves kids channel and press the Bell icon to watch new cartoon videos  subscribe to Sonic octaves kids channel and press the Bell icon to watch new cartoon videos  but then it's just me he saw just one tiny little fish in that net grab hold of that fish but then suddenly the fish started talking to him  Bank of Beaver so that when you come next time you have much more fisherman talk to himself tomorrow I will get a lot of more fish because  would bring the fish was very clever and because of his cleverness he saved his life from this fisherman so children you have to be really really clever to save your life from such challenging moments   


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
#Downloading audio
url = "https://youtu.be/QUTYxwTsbiM"


In [ ]:
ydl_opts={}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)
video_title = info_dict['title']
video_name = re.sub('[\\\\/*?:"<>|]', '', video_title)
name = video_name
ydl_opts = {
     'format': 'm4a/bestaudio/best',
         'noplaylist': True,
         'continue_dl': True,
         'outtmpl': f'./{name}.wav',
         'postprocessors': [{
             'key': 'FFmpegExtractAudio',
             'preferredcodec': 'wav',
             'preferredquality': '192',
         }],
         'geobypass':True,
         'ffmpeg_location':'/usr/bin/ffmpeg'
 }

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
     error_code = ydl.download(url)

[youtube] Extracting URL: https://youtu.be/QUTYxwTsbiM
[youtube] QUTYxwTsbiM: Downloading webpage
[youtube] QUTYxwTsbiM: Downloading android player API JSON
[youtube] Extracting URL: https://youtu.be/QUTYxwTsbiM
[youtube] QUTYxwTsbiM: Downloading webpage
[youtube] QUTYxwTsbiM: Downloading android player API JSON
[info] QUTYxwTsbiM: Downloading 1 format(s): 140
[dashsegments] Total fragments: 1
[download] Destination: ./Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav
[download] 100% of    3.27MiB in 00:00:00 at 12.75MiB/s              
[FixupM4a] Correcting container of "./Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav"
[ExtractAudio] Destination: ./Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav.wav
Deleting original file ./Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav (pass -k to keep)


In [ ]:
#Splitting audio
class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + filename
        # print(self.filepath)
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(split_filename, format="wav")
        
    def multiple_split(self, min_per_split):
        total_mins = math.ceil(self.get_duration() / 60)
        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+min_per_split, split_fn)
            if i == total_mins - min_per_split:
                print('All splited successfully')
        print('>>> Video duration: ' + str(self.get_duration()))
        
def split_audio(file_name):
    directory = "splitted files for: " + file_name
    os.mkdir(directory)
    split_wav = SplitWavAudioMubin("/content/", file_name)
    split_wav.multiple_split(min_per_split=1)

In [ ]:
file_name = "{}.wav.wav".format(video_name)
split_audio(file_name)

All splited successfully
>>> Video duration: 211.69632653061225


In [ ]:
#Recognizing text
search_dir = "./"
files = filter(os.path.isfile, os.listdir(search_dir))
files = [os.path.join(search_dir, f) for f in files] # add path to each file
files.sort(key=lambda x: os.path.getmtime(x))
files

['./Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav.wav',
 './0_Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav.wav',
 './1_Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav.wav',
 './2_Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav.wav',
 './3_Clever Fish - English Stories For Kids  Moral Stories In English  Short Story In English.wav.wav']

In [ ]:
def speech_recognizer(files, frames, i):  ## This function recognizes speech in our WAV files
    texts = []
    recognizer = sr.Recognizer()

    for file in tqdm(files):
        with sr.AudioFile(file) as source:
            recorded_audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(
                recorded_audio,
                language="en-US"  ## Replace with language keyword
            )
            texts.append(text)
        except Exception as ex:
            print(ex)
    result = ""
    for text in texts:
        result += " " + text
    frames[i] = result
    return result

def split_files(files, n_batches): ## This function split the files evenly between the threads we have.
    k, m = divmod(len(files), n_batches)
    return list(files[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n_batches))

def main(n_batches=8, verbose=True): ## By default, the maximum capacity of threads supported in collab is 8
    threads = [None]*n_batches
    frames = [None]*n_batches
    batches = split_files(files, n_batches)
    start = 0
    for i in range(len(batches)):
        if i>0:
            start_index=len(batches[i-1])
        else:
            start_index = 0
        t = Thread(target=speech_recognizer, args=(batches[i], frames, i))
        threads[i] = t
        t.start()
    for t in threads:
        t.join()
    return frames

In [ ]:
if __name__ == "__main__":
    frames = main()



0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/1 [00:00<?, ?it/s]


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]




  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]




100%|██████████| 1/1 [00:01<00:00,  1.91s/it]





100%|██████████| 1/1 [00:01<00:00,  1.97s/it]

100%|██████████| 1/1 [00:06<00:00,  6.28s/it]






100%|██████████| 1/1 [00:15<00:00, 15.96s/it]


In [ ]:
result = ""
for text in frames:
    result += " " + text

os.chdir("../")
text_file = open("Transcription_"+ file_name[:-4] +".txt", "w")
text_file.write(result)
text_file.close()

In [ ]:
result

"  subscribe to Sonic octave's kids channel and press the Bell icon to watch new cartoon videos  subscribe to Sonic octave's kids channel and press the Bell icon to watch new cartoon videos  but then do his discipline he saw just one tiny little fish in that net he grabbed hold of that fish but then suddenly the fish started talking to him tiny little fish said to the fisherman please please leave me please but fisherman did not give any attention to the request of the fish but then again the tiny little fish said to the fisherman something which is of your help if you leave me back in water I will tell all my friends about you and I will bring them here  Bank of river so that when you come next time you will have much more fish fisherman talk to himself wow that's not a bad deal at all he was thinking if I let go one time a little fish today tomorrow I will get a lot of more fish because this tiny little fish will bring all his friends to me believing the word of this tiny little fish

In [ ]:
!pip install transformers[torch]==4.3
!pip install packaging==21.3
!pip install torch
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import T5Config

In [ ]:
# initialize the pretrained model
model = T5ForConditionalGeneration.from_pretrained('t5-large')
tokenizer = T5Tokenizer.from_pretrained('t5-large')
device = torch.device('cpu')

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [ ]:
text=result
## preprocess the input text
preprocessed_text =text.strip().replace('\n','')
t5_input_text = 'summarize: ' + preprocessed_text

In [ ]:
text=result
## preprocess the input text
preprocessed_text =text.strip().replace('\n','')
t5_input_text = 'summarize: ' + preprocessed_text
ml=len(t5_input_text.split())
tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=ml).to(device)
summary_ids = model.generate(tokenized_text, min_length=30, max_length=200)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
summary

"subscribe to Sonic octave's kids channel and press the Bell icon to watch new cartoon videos. tiny little fish said to the fisherman please please leave me please but fisherman did not give any attention to the request of the fish. fisherman let go the style of fish into the river again this tiny little fish was really happy and it's swam away happily into the river never to come back."

In [ ]:
t5_input_text

"summarize: subscribe to Sonic octave's kids channel and press the Bell icon to watch new cartoon videos  subscribe to Sonic octave's kids channel and press the Bell icon to watch new cartoon videos  but then do his discipline he saw just one tiny little fish in that net he grabbed hold of that fish but then suddenly the fish started talking to him tiny little fish said to the fisherman please please leave me please but fisherman did not give any attention to the request of the fish but then again the tiny little fish said to the fisherman something which is of your help if you leave me back in water I will tell all my friends about you and I will bring them here  Bank of river so that when you come next time you will have much more fish fisherman talk to himself wow that's not a bad deal at all he was thinking if I let go one time a little fish today tomorrow I will get a lot of more fish because this tiny little fish will bring all his friends to me believing the word of this tiny li

In [ ]:
ml=len(t5_input_text.split())

In [ ]:
tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=ml).to(device)

In [ ]:
summary_ids = model.generate(tokenized_text, min_length=30, max_length=200)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
summary

"subscribe to Sonic octave's kids channel and press the Bell icon to watch new cartoon videos. tiny little fish said to the fisherman please please leave me please but fisherman did not give any attention to the request of the fish. fisherman let go the style of fish into the river again this tiny little fish was really happy and it's swam away happily into the river never to come back."

In [ ]:
# Import the required module for text
# to speech conversion
!pip install gtts
from gtts import gTTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# This module is imported so that we can
# play the converted audio
import os

# The text that you want to convert to audio
mytext = summary

# Language in which you want to convert
language = 'en'

# Passing the text and language to the engine,
# here we have marked slow=False. Which tells
# the module that the converted audio should
# have a high speed
myobj = gTTS(text=mytext, lang=language, slow=False)

# Saving the converted audio in a mp3 file named
# welcome
myobj.save("welcome.mp3")

# Playing the converted file
os.system("mpg321 welcome.mp3")


32512

In [ ]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge

In [ ]:
summary="tiny little fish said to the fisherman please please leave me please but fisherman did not give any attention to the request of the fish. fisherman let go the style of fish into the river again this tiny little fish was really happy and it's swam away happily into the river never to come back."

In [ ]:
reference=" Fisherman saw just one tiny little fish in that net he grabbed hold of that fish but then suddenly the fish started talking to him. The fish said to the fisherman that if you leave me back in water I will tell all my friends about you and I will bring them here  Bank of river so that when you come next time you will have much more fish. Fisherman thought that if I let go one time a little fish today tomorrow I will get a lot of more fish because this tiny little. fish will bring all his friends to me believing the word of this tiny. little fish fisherman let go the style of fish into the river again believing the fish."

In [ ]:
rouge = Rouge()
dic=rouge.get_scores(summary, reference)

In [ ]:
dic


[{'rouge-1': {'r': 0.3088235294117647,
   'p': 0.5833333333333334,
   'f': 0.40384614931952667},
  'rouge-2': {'r': 0.18181818181818182,
   'p': 0.40816326530612246,
   'f': 0.25157232277995334},
  'rouge-l': {'r': 0.29411764705882354,
   'p': 0.5555555555555556,
   'f': 0.3846153800887574}}]

In [ ]:
import json
type(json.dumps(dic))

str

In [ ]:
type(dic)

list

In [ ]:
res=[''.join(ele) for ele in dic]
res

['rouge-1rouge-2rouge-l']